In [4]:
import pandas as pd
import numpy as np

In [6]:
train_df = pd.read_csv('PlayTennis.csv')
train_df.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [8]:
# entropy formula
# - p(x) * log_2(p(x))
# total count = 14
# Yes = 9 & No = 5

# Full entropy
- ((9/14) * np.log2(9/14)) - ((5/14) * np.log2(5/14))

0.9402859586706311

In [10]:
def total_entropy(feature_name, df):
    values = pd.unique(df[feature_name])
    sum_entropy = 0
    total = len(df)
    
    for i in values:
        value_count = sum(df[feature_name] == i)
        sum_entropy -= (value_count/total) * np.log2(value_count/total)
    return sum_entropy

In [12]:
total_entropy('Play Tennis', train_df)

0.9402859586706311

In [14]:
pd.unique(train_df['Play Tennis'])

array(['No', 'Yes'], dtype=object)

In [16]:
train_df[(train_df.Outlook == 'Sunny') & (train_df['Play Tennis'] == 'Yes')]

,Outlook,Temperature,Humidity,Wind,Play Tennis
8,Sunny,Cool,Normal,Weak,Yes
10,Sunny,Mild,Normal,Strong,Yes


In [18]:
def binary_entropy(feature_name, df, label):
    values = pd.unique(df[feature_name])
    sum_entropy = 0
    total = len(df)
    unique_labels = pd.unique(df[label])

    # Different values in a column
    for i in values:
        value_count = sum(df[feature_name] == i)
        temp_df = df[df[feature_name] == i]
        #Yes or No
        value_sum = 0
        for j in unique_labels:
            # -(2/5)*log(2/5)-(3/5)*log(3/5) = 0.971
            selected_df = temp_df[temp_df[label] == j]
            selected_value = len(selected_df)
            if selected_value == 0 or selected_value == value_count:
                break
            value_sum -= (selected_value/value_count) * np.log2(selected_value/value_count)
        sum_entropy += value_sum
            
    return sum_entropy

In [20]:
def binary_entropy_single(feature_name, wanted_value, df, label):
    values = pd.unique(df[feature_name])
    sum_entropy = 0
    total = len(df)
    unique_labels = pd.unique(df[label])

    # Different values in a column
    temp_df = df[df[feature_name] == wanted_value]
    value_count = len(temp_df)
    #Yes or No
    value_sum = 0
    for j in unique_labels:
        # -(2/5)*log(2/5)-(3/5)*log(3/5) = 0.971
        selected_df = temp_df[temp_df[label] == j]
        selected_value = len(selected_df)
        if selected_value == 0 or selected_value == value_count:
            break
        value_sum -= (selected_value/value_count) * np.log2(selected_value/value_count)
    sum_entropy += value_sum
            
    return sum_entropy

In [22]:
binary_entropy_single('Outlook', 'Sunny', train_df, 'Play Tennis')

0.9709505944546686

In [24]:
def info(feature_name, df, label):
    values = pd.unique(df[feature_name])
    size = len(df)
    sum = 0
    
    for i in values:
        h = binary_entropy_single(feature_name, i, train_df, label)
        filter = df[df[feature_name] == i]
        p = (len(filter)/size)
        sum += p * h
    return sum

In [26]:
train_df.columns

Index(['Outlook', 'Temperature', 'Humidity', 'Wind', 'Play Tennis'], dtype='object')

In [49]:
def informative_feature(train_df, label):
    entropy = total_entropy(label, train_df)
    col_inf = {}
    features = train_df.columns.drop(label)
    max_inf = "NO VALUE"
    max_value = 0
    for i in features:
        
        inf = info(i, train_df, label)
        information_gain = entropy - inf
        if information_gain > max_value:
            max_value = information_gain
            max_inf = i
        # print(f'For column {i} the information gain is: {information_gain:.3f}')
        col_inf[i] = information_gain

    return max_inf, max_value, col_inf

In [53]:
x, _, _= informative_feature(train_df, 'Play Tennis')
x

'Outlook'

In [68]:
def make_feature_tree(feature_name, data, label):
    unique_features = pd.unique(data[feature_name])
    tree = {}
    unique_labels = pd.unique(data[label])
    for value in unique_features:
        filtered_value = data[data[feature_name] == value] 
        pure = False 
        for i in unique_labels: 
            label_count = filtered_value[filtered_value[label] == i].shape[0]
            if label_count == len(filtered_value): 
                tree[value] = i
                data = data[data[feature_name] != value]
                pure = True
        if not pure:
            tree[value] = "?"
            
    return tree, data

In [109]:
t, d = make_feature_tree('Humidity', train_df, 'Play Tennis')
make_feature_tree('Humidity', d, 'Play Tennis')

({'High': '?', 'Normal': '?'},
      Outlook Temperature Humidity    Wind Play Tennis
 0      Sunny         Hot     High    Weak          No
 1      Sunny         Hot     High  Strong          No
 2   Overcast         Hot     High    Weak         Yes
 3       Rain        Mild     High    Weak         Yes
 4       Rain        Cool   Normal    Weak         Yes
 5       Rain        Cool   Normal  Strong          No
 6   Overcast        Cool   Normal  Strong         Yes
 7      Sunny        Mild     High    Weak          No
 8      Sunny        Cool   Normal    Weak         Yes
 9       Rain        Mild   Normal    Weak         Yes
 10     Sunny        Mild   Normal  Strong         Yes
 11  Overcast        Mild     High  Strong         Yes
 12  Overcast         Hot   Normal    Weak         Yes
 13      Rain        Mild     High  Strong          No)

### To-Do:
- Finding the most informative feature: **informative_feature()**
- Making a tree node with a feature name and feature values as branches: **make_feature_tree()**
    - If pure class, adding leaf node (= Class) to the tree node
    - If impure class, adding an expandable node (= ‘?’) to the tree node
- Shrinking/Updating the dataset according to the pure class
- Adding the node with branches into a tree
- Expand the branch of the next impure class (= ‘?’) with an updated dataset

In [129]:
def make_tree(root, prev_feature_value, train_df, label):
    if len(train_df) != 0:
        best_feature_name, _, _ = informative_feature(train_df, label)
        print(best_feature_name)
        print(train_df)
        tree, train_df = make_feature_tree(best_feature_name, train_df, label)
        print(tree)

        next_root = None

        # Initial
        if prev_feature_value != None:
            root[prev_feature_value] = dict()
            root[prev_feature_value][best_feature_name] = tree
            next_root = root[prev_feature_value][best_feature_name]
        else:
            root[best_feature_name] = tree
            next_root = root[best_feature_name]
        for node, branch in list(next_root.items()):
            print()
            print(node, branch)
            if branch == "?":
                feature_value_data = train_df[train_df[best_feature_name] == node]
                make_tree(next_root, node, feature_value_data, label) 

In [131]:
def id3(train_data_m, label):
    train_data = train_df.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated
    make_tree(tree, None, train_data, label) #start calling recursion
    return tree

In [133]:
tree = id3(train_df, 'Play Tennis')

Outlook
     Outlook Temperature Humidity    Wind Play Tennis
0      Sunny         Hot     High    Weak          No
1      Sunny         Hot     High  Strong          No
2   Overcast         Hot     High    Weak         Yes
3       Rain        Mild     High    Weak         Yes
4       Rain        Cool   Normal    Weak         Yes
5       Rain        Cool   Normal  Strong          No
6   Overcast        Cool   Normal  Strong         Yes
7      Sunny        Mild     High    Weak          No
8      Sunny        Cool   Normal    Weak         Yes
9       Rain        Mild   Normal    Weak         Yes
10     Sunny        Mild   Normal  Strong         Yes
11  Overcast        Mild     High  Strong         Yes
12  Overcast         Hot   Normal    Weak         Yes
13      Rain        Mild     High  Strong          No
{'Sunny': '?', 'Overcast': 'Yes', 'Rain': '?'}

Sunny ?
Humidity
   Outlook Temperature Humidity    Wind Play Tennis
0    Sunny         Hot     High    Weak          No
1    Sunny   

KeyboardInterrupt: 

In [ ]:
def predict(tree, instance):
    if not isinstance(tree, dict):
        return tree
    else:
        root_node = next(iter(tree))
        feature_value = instance[root_node] 
        if feature_value in tree[root_node]:
            return predict(tree[root_node][feature_value], instance)
        else:
            return None